# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [1]:
%pip install iso3166

Note: you may need to restart the kernel to use updated packages.


### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [53]:
%pip install --upgrade nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


### Import Statements

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [3]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [4]:
#df_data.shape

#df_data.info()

#Are there any NaN Values? or Duplicates?
df_data.isna().sum()   # there are 3360 NaN in the price columnS
#df_data.duplicated().sum()

Unnamed: 0.1         0
Unnamed: 0           0
Organisation         0
Location             0
Date                 0
Detail               0
Rocket_Status        0
Price             3360
Mission_Status       0
dtype: int64

In [5]:
#How many duplicates are there in the dataset?
df_data.duplicated().sum()   # There are 0 duplicatesS

np.int64(0)

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [4]:
#Check for missing values in the dataset, then remove columns that contains junk data
df_data.isna().sum()   # there are 3360 NaN in the price columnS
# drop junk columns if they exist (avoid KeyError if columns are missing)
df_data = df_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], errors='ignore')

df_data.head()


,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [5]:
# Who launched the most missions in each year?

# Step 1: Parse Date and extract Year
df_data['Date'] = pd.to_datetime(df_data['Date'], errors='coerce')  # invalid dates -> NaT
df_data['Year'] = df_data['Date'].dt.year

# Step 2: Drop rows without a valid year and convert to int
df_clean = df_data.dropna(subset=['Year']).copy()
df_clean['Year'] = df_clean['Year'].astype(int)

# Step 3: Count missions per (Year, Organisation)
missions_per_year = (
    df_clean
    .groupby(['Year', 'Organisation'], dropna=False)
    .size()
    .reset_index(name='Missions_Count')
)

# Step 4: Find the organisation(s) with the most missions each year
# Option A: Single winner per year (first org if there's a tie)
idx = missions_per_year.groupby('Year')['Missions_Count'].idxmax()
top_missions_per_year = missions_per_year.loc[idx].sort_values('Year').reset_index(drop=True)

# Option B: All organisations that tie for the top count each year
max_count = missions_per_year.groupby('Year')['Missions_Count'].transform('max')
top_missions_per_year_all_ties = (
    missions_per_year[missions_per_year['Missions_Count'] == max_count]
    .sort_values(['Year', 'Organisation'])
    .reset_index(drop=True)
)

# Display results
print("Single winner per year (first org if tied):")
display(top_missions_per_year.head(20))

print("\n All winners including ties:")
display(top_missions_per_year_all_ties.head(40))


Single winner per year (first org if tied):


,Year,Organisation,Missions_Count
0,1957,RVSN USSR,2
1,1958,AMBA,7
2,1959,US Air Force,10
3,1960,US Air Force,21
4,1961,US Air Force,23
5,1962,US Air Force,40
6,1963,RVSN USSR,23
7,1964,RVSN USSR,33
8,1965,RVSN USSR,52
9,1966,RVSN USSR,50



 All winners including ties:


,Year,Organisation,Missions_Count
0,1957,RVSN USSR,2
1,1958,AMBA,7
2,1959,US Air Force,10
3,1960,US Air Force,21
4,1961,US Air Force,23
5,1962,US Air Force,40
6,1963,RVSN USSR,23
7,1964,RVSN USSR,33
8,1965,RVSN USSR,52
9,1966,RVSN USSR,50


## Descriptive Statistics

In [9]:
# How has the cost of space missions changed over time?

# Create a clean copy to avoid modifying the original df_data
df_cost = df_data.copy()

# Ensure Date is datetime and drop rows missing Date or Price
df_cost['Date'] = pd.to_datetime(df_cost['Date'], errors='coerce')
df_cost = df_cost.dropna(subset=['Date', 'Price'])

# Ensure Price is numeric (convert any strings to numbers)
df_cost['Price'] = pd.to_numeric(df_cost['Price'], errors='coerce')
df_cost = df_cost.dropna(subset=['Price'])  # Drop any rows where Price couldn't be converted

df_cost['Year'] = df_cost['Date'].dt.year.astype(int)

# Calculate average cost per year
cost_per_year = (
    df_cost
    .groupby('Year')['Price']
    .mean()
    .reset_index()
    .sort_values('Year')
)

# Get top 3 organizations per year (from the missions data)
# Use df_clean from the previous cell which already has Year parsed
top_orgs_per_year = (
    df_clean
    .groupby(['Year', 'Organisation'])
    .size()
    .reset_index(name='Launch_Count')
    .sort_values(['Year', 'Launch_Count'], ascending=[True, False])
    .groupby('Year')
    .head(3)  # Get top 3 per year
)

# Create a formatted string with top 3 orgs for each year
top_orgs_text = (
    top_orgs_per_year
    .groupby('Year')
    .apply(lambda x: '<br>'.join([f"{i+1}. {row['Organisation']} ({row['Launch_Count']} launches)" 
                                   for i, row in x.iterrows()]))
    .reset_index(name='Top_Orgs')
)

# Merge top orgs with cost data
cost_per_year = cost_per_year.merge(top_orgs_text, on='Year', how='left')

# Create an enhanced line chart
fig = px.line(
    cost_per_year, 
    x='Year', 
    y='Price', 
    title='Average Cost of Space Missions Over Time (with Top Launchers)',
    labels={'Price': 'Average Price (USD Millions)', 'Year': 'Year'},
    markers=True,
    hover_data={'Top_Orgs': True, 'Year': True, 'Price': ':.2f'}
)

fig.update_traces(
    hovertemplate='<b>Year: %{x}</b><br>' +
                  'Average Price: $%{y:.2f}M<br>' +
                  '<b>Top Launchers:</b><br>%{customdata[0]}<extra></extra>'
)

fig.update_layout(
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

# Show summary statistics
print(f"📊 Cost Analysis Summary:")
print(f"   Years covered: {cost_per_year['Year'].min()} to {cost_per_year['Year'].max()}")
print(f"   Number of years analyzed: {len(cost_per_year)}")
print(f"   Overall average cost: ${cost_per_year['Price'].mean():.2f}M")
print(f"   Lowest average cost: ${cost_per_year['Price'].min():.2f}M in {cost_per_year.loc[cost_per_year['Price'].idxmin(), 'Year']:.0f}")
print(f"   Highest average cost: ${cost_per_year['Price'].max():.2f}M in {cost_per_year.loc[cost_per_year['Price'].idxmax(), 'Year']:.0f}")


/tmp/ipykernel_71258/2658211494.py:41: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



📊 Cost Analysis Summary:
   Years covered: 1964 to 2020
   Number of years analyzed: 46
   Overall average cost: $182.49M
   Lowest average cost: $30.80M in 1987
   Highest average cost: $450.00M in 1981


In [11]:
# Which months are the most popular for space missions?

# Extract month from Date and create month names
df_month = df_data.dropna(subset=['Date']).copy()
df_month['Month'] = df_month['Date'].dt.month
df_month['Month_Name'] = df_month['Date'].dt.month_name()

# Count missions per month
month_counts = (
    df_month
    .groupby(['Month', 'Month_Name'])
    .size()
    .reset_index(name='Mission_Count')
    .sort_values('Month')
)

# Calculate percentages
month_counts['Percentage'] = (month_counts['Mission_Count'] / month_counts['Mission_Count'].sum() * 100)

# Create an interactive donut chart
fig = px.pie(
    month_counts,
    values='Mission_Count',
    names='Month_Name',
    title='🚀 Space Mission Launches by Month (All Time)',
    hole=0.4,  # Creates the donut effect
    color_discrete_sequence=px.colors.sequential.Blues_r,
    hover_data={'Mission_Count': ':,', 'Percentage': ':.1f'}
)

fig.update_traces(
    textposition='outside',
    textinfo='label+percent',
    hovertemplate='<b>%{label}</b><br>' +
                  'Missions: %{value:,}<br>' +
                  'Percentage: %{customdata[1]:.1f}%<extra></extra>'
)

fig.update_layout(
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.1
    ),
    height=600
)

fig.show()

# Show statistics
print(f"\n📊 Monthly Launch Statistics:")
print(f"   Most popular month: {month_counts.loc[month_counts['Mission_Count'].idxmax(), 'Month_Name']} "
      f"({month_counts['Mission_Count'].max():,} launches)")
print(f"   Least popular month: {month_counts.loc[month_counts['Mission_Count'].idxmin(), 'Month_Name']} "
      f"({month_counts['Mission_Count'].min():,} launches)")
print(f"   Average per month: {month_counts['Mission_Count'].mean():.0f} launches")

# Additional: Create a season analysis
season_map = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
}
df_month['Season'] = df_month['Month'].map(season_map)
season_counts = df_month['Season'].value_counts()

print(f"\n🌍 Seasonal Distribution:")
for season, count in season_counts.items():
    pct = (count / season_counts.sum() * 100)
    print(f"   {season}: {count:,} launches ({pct:.1f}%)")



📊 Monthly Launch Statistics:
   Most popular month: December (430 launches)
   Least popular month: January (265 launches)
   Average per month: 350 launches

🌍 Seasonal Distribution:
   Summer: 1,091 launches (26.0%)
   Fall: 1,054 launches (25.1%)
   Spring: 1,035 launches (24.7%)
   Winter: 1,018 launches (24.2%)


In [20]:
# Visualize monthly and seasonal statistics with bar charts

# Create a figure with two subplots side by side
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('📊 Launches by Month', '🌍 Launches by Season'),
    specs=[[{"type": "bar"}, {"type": "bar"}]],
    horizontal_spacing=0.15
)

# Monthly bar chart
fig.add_trace(
    go.Bar(
        x=month_counts['Month_Name'],
        y=month_counts['Mission_Count'],
        marker=dict(
            color=month_counts['Mission_Count'],
            colorscale='Blues',
            showscale=False,
            line=dict(color='rgb(8,48,107)', width=1.5)
        ),
        text=month_counts['Mission_Count'],
        textposition='outside',
        texttemplate='%{text:,}',
        hovertemplate='<b>%{x}</b><br>Launches: %{y:,}<extra></extra>',
        name='Monthly'
    ),
    row=1, col=1
)

# Seasonal bar chart (need to prepare data)
season_df = pd.DataFrame({
    'Season': season_counts.index,
    'Count': season_counts.values
})
# Order seasons logically
season_order = ['Spring', 'Summer', 'Fall', 'Winter']
season_df['Season'] = pd.Categorical(season_df['Season'], categories=season_order, ordered=True)
season_df = season_df.sort_values('Season')

# Color mapping for seasons
season_colors = {
    'Spring': '#90EE90',  # Light green
    'Summer': '#FFD700',  # Gold
    'Fall': '#FF8C00',    # Dark orange
    'Winter': '#4682B4'   # Steel blue
}

fig.add_trace(
    go.Bar(
        x=season_df['Season'],
        y=season_df['Count'],
        marker=dict(
            color=[season_colors[s] for s in season_df['Season']],
            line=dict(color='rgb(8,48,107)', width=1.5)
        ),
        text=season_df['Count'],
        textposition='outside',
        texttemplate='%{text:,}',
        hovertemplate='<b>%{x}</b><br>Launches: %{y:,}<br>Percentage: %{customdata:.1f}%<extra></extra>',
        customdata=[(c / season_counts.sum() * 100) for c in season_df['Count']],
        name='Seasonal'
    ),
    row=1, col=2
)

# Update layout
fig.update_xaxes(title_text="Month", row=1, col=1, tickangle=-45)
fig.update_xaxes(title_text="Season", row=1, col=2)
fig.update_yaxes(title_text="Number of Launches", row=1, col=1)
fig.update_yaxes(title_text="Number of Launches", row=1, col=2)

fig.update_layout(
    title_text="Space Mission Launch Patterns: Monthly and Seasonal Distribution",
    title_font_size=16,
    showlegend=False,
    height=500,
    template='plotly_white'
)

fig.show()

# Print the statistics alongside
print(f"\n📊 Monthly Launch Statistics:")
print(f"   Most popular month: {month_counts.loc[month_counts['Mission_Count'].idxmax(), 'Month_Name']} "
      f"({month_counts['Mission_Count'].max():,} launches)")
print(f"   Least popular month: {month_counts.loc[month_counts['Mission_Count'].idxmin(), 'Month_Name']} "
      f"({month_counts['Mission_Count'].min():,} launches)")
print(f"   Average per month: {month_counts['Mission_Count'].mean():.0f} launches")
print(f"   Range: {month_counts['Mission_Count'].max() - month_counts['Mission_Count'].min()} launches")

print(f"\n🌍 Seasonal Distribution:")
for season in season_order:
    if season in season_counts.index:
        count = season_counts[season]
        pct = (count / season_counts.sum() * 100)
        print(f"   {season}: {count:,} launches ({pct:.1f}%)")
print(f"   Most active season: {season_counts.idxmax()}")
print(f"   Least active season: {season_counts.idxmin()}")



📊 Monthly Launch Statistics:
   Most popular month: December (430 launches)
   Least popular month: January (265 launches)
   Average per month: 350 launches
   Range: 165 launches

🌍 Seasonal Distribution:
   Spring: 1,035 launches (24.7%)
   Summer: 1,091 launches (26.0%)
   Fall: 1,054 launches (25.1%)
   Winter: 1,018 launches (24.2%)
   Most active season: Summer
   Least active season: Winter


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [13]:
# Choropleth Map: Number of Launches by Country

# Step 1: Extract country from Location column
df_map = df_data.copy()

# Step 2: Wrangle country names - handle special cases
location_to_country = {
    'Russia': 'Russian Federation',
    'New Mexico': 'USA',
    'Yellow Sea': 'China',
    'Shahrud Missile Test Site': 'Iran',
    'Pacific Missile Range Facility': 'USA',
    'Barents Sea': 'Russian Federation',
    'Gran Canaria': 'USA',
    # Additional common mappings
    'Pacific Ocean': 'USA',
    'Barents Sea, Russia': 'Russian Federation',
    'Pacific Spaceport Complex': 'USA'
}

# Extract the last part of location (usually the country)
def extract_country(location):
    if pd.isna(location):
        return None
    
    # Check if it's in our special mapping
    for key, value in location_to_country.items():
        if key in str(location):
            return value
    
    # Split by comma and take the last part (usually country)
    parts = str(location).split(',')
    country = parts[-1].strip()
    
    # Handle some common patterns
    if 'USA' in location or 'United States' in location:
        return 'USA'
    elif 'Kazakhstan' in location:
        return 'Kazakhstan'
    elif 'China' in location:
        return 'China'
    elif 'Russia' in location or 'USSR' in location:
        return 'Russian Federation'
    elif 'French Guiana' in location:
        return 'France'
    
    return country

df_map['Country'] = df_map['Location'].apply(extract_country)

# Step 3: Convert country names to ISO Alpha-3 codes
def get_alpha3_code(country_name):
    if pd.isna(country_name):
        return None
    
    try:
        # Special cases that iso3166 might not recognize
        special_codes = {
            'USA': 'USA',
            'Russian Federation': 'RUS',
            'Iran': 'IRN',
            'South Korea': 'KOR',
            'North Korea': 'PRK',
            'New Zealand': 'NZL',
            'Kazakhstan': 'KAZ'
        }
        
        if country_name in special_codes:
            return special_codes[country_name]
        
        # Try iso3166 package
        country = countries.get(country_name)
        return country.alpha3
    except:
        # If all else fails, return the original name
        return None

df_map['Country_Code'] = df_map['Country'].apply(get_alpha3_code)

# Step 4: Count launches per country
country_launches = (
    df_map
    .dropna(subset=['Country_Code'])
    .groupby(['Country', 'Country_Code'])
    .size()
    .reset_index(name='Launch_Count')
    .sort_values('Launch_Count', ascending=False)
)

# Step 5: Create the choropleth map
fig = px.choropleth(
    country_launches,
    locations='Country_Code',
    color='Launch_Count',
    hover_name='Country',
    hover_data={'Country_Code': False, 'Launch_Count': ':,'},
    color_continuous_scale='matter',  # Using the requested colorscale
    title='🌍 Number of Space Mission Launches by Country (All Time)',
    labels={'Launch_Count': 'Number of Launches'}
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600,
    coloraxis_colorbar=dict(
        title="Launches",
        thickness=15,
        len=0.7
    )
)

fig.show()

# Display top 10 countries
print("\n🚀 Top 10 Countries by Number of Launches:")
print(country_launches.head(10).to_string(index=False))

print(f"\n📊 Launch Statistics by Country:")
print(f"   Total countries: {len(country_launches)}")
print(f"   Top launcher: {country_launches.iloc[0]['Country']} ({country_launches.iloc[0]['Launch_Count']:,} launches)")
print(f"   Total launches mapped: {country_launches['Launch_Count'].sum():,}")



🚀 Top 10 Countries by Number of Launches:
           Country Country_Code  Launch_Count
Russian Federation          RUS          1398
               USA          USA          1387
        Kazakhstan          KAZ           701
            France          FRA           303
             China          CHN           269
             Japan          JPN           126
             India          IND            76
              Iran          IRN            14
       New Zealand          NZL            13
            Israel          ISR            11

📊 Launch Statistics by Country:
   Total countries: 15
   Top launcher: Russian Federation (1,398 launches)
   Total launches mapped: 4,324


# Use a Choropleth Map to Show the Number of Failures by Country


In [15]:
# Choropleth Map: Number of Mission Failures by Country

# Filter for failed missions
df_failures = df_map[df_map['Mission_Status'].str.contains('Failure', case=False, na=False)].copy()

# Count failures per country
country_failures = (
    df_failures
    .dropna(subset=['Country_Code'])
    .groupby(['Country', 'Country_Code'])
    .size()
    .reset_index(name='Failure_Count')
    .sort_values('Failure_Count', ascending=False)
)

# Merge with total launches to calculate failure rate
country_stats = country_launches.merge(
    country_failures, 
    on=['Country', 'Country_Code'], 
    how='left'
)
country_stats['Failure_Count'] = country_stats['Failure_Count'].fillna(0).astype(int)
country_stats['Failure_Rate'] = (country_stats['Failure_Count'] / country_stats['Launch_Count'] * 100).round(2)

# Create the choropleth map for failures
fig = px.choropleth(
    country_failures,
    locations='Country_Code',
    color='Failure_Count',
    hover_name='Country',
    hover_data={'Country_Code': False, 'Failure_Count': ':,'},
    color_continuous_scale='Reds',  # Red color scheme for failures
    title='❌ Number of Mission Failures by Country (All Time)',
    labels={'Failure_Count': 'Number of Failures'}
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600,
    coloraxis_colorbar=dict(
        title="Failures",
        thickness=15,
        len=0.7
    )
)

fig.show()

# Display top 10 countries by failures and failure rates
print("\n❌ Top 10 Countries by Number of Failures:")
print(country_failures.head(10).to_string(index=False))

print("\n📊 Failure Rate Analysis (Countries with 10+ launches):")
high_launch_countries = country_stats[country_stats['Launch_Count'] >= 10].copy()
high_launch_countries = high_launch_countries.sort_values('Failure_Rate', ascending=False)
print(high_launch_countries[['Country', 'Launch_Count', 'Failure_Count', 'Failure_Rate']].head(10).to_string(index=False))

print(f"\n📈 Overall Statistics:")
print(f"   Total failures mapped: {country_failures['Failure_Count'].sum():,}")
print(f"   Average failure rate: {country_stats['Failure_Rate'].mean():.2f}%")
print(f"   Country with most failures: {country_failures.iloc[0]['Country']} ({country_failures.iloc[0]['Failure_Count']:,} failures)")



❌ Top 10 Countries by Number of Failures:
           Country Country_Code  Failure_Count
               USA          USA            166
        Kazakhstan          KAZ             93
Russian Federation          RUS             93
             China          CHN             25
            France          FRA             18
             India          IND             13
             Japan          JPN             13
              Iran          IRN              9
         Australia          AUS              3
            Brazil          BRA              3

📊 Failure Rate Analysis (Countries with 10+ launches):
           Country  Launch_Count  Failure_Count  Failure_Rate
              Iran            14              9         64.29
            Israel            11              2         18.18
             India            76             13         17.11
       New Zealand            13              2         15.38
        Kazakhstan           701             93         13.27
            

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [17]:
# Create a Sunburst Chart: Country → Organisation → Mission Status

# Prepare data for sunburst chart
df_sunburst = df_map.dropna(subset=['Country', 'Organisation', 'Mission_Status']).copy()

# Clean up mission status for better display
df_sunburst['Mission_Status_Clean'] = df_sunburst['Mission_Status'].apply(
    lambda x: 'Success' if 'Success' in str(x) else 
             ('Failure' if 'Failure' in str(x) else 
             ('Partial Failure' if 'Partial' in str(x) else 'Other'))
)

# Count missions for each combination
sunburst_data = (
    df_sunburst
    .groupby(['Country', 'Organisation', 'Mission_Status_Clean'])
    .size()
    .reset_index(name='Count')
)

# Filter to top 10 countries and their top organizations to keep chart readable
top_countries = (
    df_sunburst
    .groupby('Country')
    .size()
    .nlargest(10)
    .index
)

# For each top country, keep only top 10 organizations
filtered_data = []
for country in top_countries:
    country_data = sunburst_data[sunburst_data['Country'] == country]
    top_orgs = (
        country_data
        .groupby('Organisation')['Count']
        .sum()
        .nlargest(10)
        .index
    )
    filtered_data.append(country_data[country_data['Organisation'].isin(top_orgs)])

sunburst_filtered = pd.concat(filtered_data, ignore_index=True)

# Create the sunburst chart
fig = px.sunburst(
    sunburst_filtered,
    path=['Country', 'Organisation', 'Mission_Status_Clean'],
    values='Count',
    title='🌞 Space Missions Sunburst: Country → Organisation → Mission Status',
    color='Mission_Status_Clean',
    color_discrete_map={
        'Success': '#2ecc71',      # Green
        'Failure': '#e74c3c',      # Red
        'Partial Failure': '#f39c12',  # Orange
        'Other': '#95a5a6'         # Gray
    },
    hover_data={'Count': ':,'}
)

fig.update_traces(
    textinfo='label+percent parent',
    hovertemplate='<b>%{label}</b><br>Missions: %{value:,}<br>Percentage: %{percentParent:.1%}<extra></extra>'
)

fig.update_layout(
    height=800,
    margin=dict(t=100, l=0, r=0, b=0)
)

fig.show()

# Display summary statistics
print("\n📊 Sunburst Chart Summary:")
print(f"   Countries displayed: {sunburst_filtered['Country'].nunique()}")
print(f"   Organizations displayed: {sunburst_filtered['Organisation'].nunique()}")
print(f"   Total missions shown: {sunburst_filtered['Count'].sum():,}")

print("\n🎯 Mission Status Breakdown:")
status_summary = sunburst_filtered.groupby('Mission_Status_Clean')['Count'].sum().sort_values(ascending=False)
for status, count in status_summary.items():
    pct = (count / status_summary.sum() * 100)
    print(f"   {status}: {count:,} missions ({pct:.1f}%)")

print("\n🌍 Top 5 Countries in Chart:")
country_summary = sunburst_filtered.groupby('Country')['Count'].sum().sort_values(ascending=False).head(5)
for country, count in country_summary.items():
    print(f"   {country}: {count:,} missions")



📊 Sunburst Chart Summary:
   Countries displayed: 10
   Organizations displayed: 42
   Total missions shown: 4,218

🎯 Mission Status Breakdown:
   Success: 3,813 missions (90.4%)
   Failure: 405 missions (9.6%)

🌍 Top 5 Countries in Chart:
   Russian Federation: 1,398 missions
   USA: 1,307 missions
   Kazakhstan: 701 missions
   France: 303 missions
   China: 269 missions


# Analyse the Total Amount of Money Spent by Organisation on Space Missions

# Analyse the Amount of Money Spent by Organisation per Launch

# Chart the Number of Launches per Year

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [19]:
# Year-on-Year Chart: Organisation with Most Launches per Year

# Get top 10 organizations overall
top_10_orgs = (
    df_clean
    .groupby('Organisation')
    .size()
    .nlargest(10)
    .index
)

# Get launch counts per year for top 10 organizations
df_top10 = df_clean[df_clean['Organisation'].isin(top_10_orgs)].copy()
launches_by_year_org = (
    df_top10
    .groupby(['Year', 'Organisation'])
    .size()
    .reset_index(name='Launch_Count')
)

# Create an interactive area chart showing dominance over time
fig = px.area(
    launches_by_year_org,
    x='Year',
    y='Launch_Count',
    color='Organisation',
    title='🚀 Space Launch Dominance Over Time - Top 10 Organizations',
    labels={'Launch_Count': 'Number of Launches per Year', 'Year': 'Year'},
    height=700
)

fig.update_layout(
    hovermode='x unified',
    template='plotly_white',
    xaxis=dict(
        rangeslider=dict(visible=True, thickness=0.05),
        type='linear'
    ),
    legend=dict(
        title='Organization',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.show()

# Answer the specific questions with detailed analysis
print("\n🏆 Dominant Organizations by Decade:\n")

# 1970s
seventies_data = df_top10[(df_top10['Year'] >= 1970) & (df_top10['Year'] < 1980)]
seventies_summary = seventies_data.groupby('Organisation').size().sort_values(ascending=False)
print("📅 1970s (1970-1979):")
for org, count in seventies_summary.head(5).items():
    print(f"   {org}: {count} launches")
if len(seventies_summary) > 0:
    print(f"   🥇 Most active: {seventies_summary.idxmax()} ({seventies_summary.max()} launches)")

# 1980s
eighties_data = df_top10[(df_top10['Year'] >= 1980) & (df_top10['Year'] < 1990)]
eighties_summary = eighties_data.groupby('Organisation').size().sort_values(ascending=False)
print("\n📅 1980s (1980-1989):")
for org, count in eighties_summary.head(5).items():
    print(f"   {org}: {count} launches")
if len(eighties_summary) > 0:
    print(f"   🥇 Most active: {eighties_summary.idxmax()} ({eighties_summary.max()} launches)")

# 2018-2020
recent_data = df_top10[(df_top10['Year'] >= 2018) & (df_top10['Year'] <= 2020)]
recent_by_year = recent_data.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')
print("\n📅 Recent Years (2018-2020):")
for year in [2018, 2019, 2020]:
    year_data = recent_by_year[recent_by_year['Year'] == year].sort_values('Launches', ascending=False)
    if len(year_data) > 0:
        top_org = year_data.iloc[0]
        print(f"   {year}: {top_org['Organisation']} - {top_org['Launches']} launches")
        if len(year_data) > 1:
            for _, row in year_data.iloc[1:3].iterrows():
                print(f"         {row['Organisation']} - {row['Launches']} launches")

# Overall summary
print("\n📊 All-Time Top 10 Organizations (Total Launches):")
overall_summary = df_top10.groupby('Organisation').size().sort_values(ascending=False)
for i, (org, count) in enumerate(overall_summary.items(), 1):
    print(f"   {i}. {org}: {count:,} launches")



🏆 Dominant Organizations by Decade:

📅 1970s (1970-1979):
   RVSN USSR: 789 launches
   General Dynamics: 55 launches
   Martin Marietta: 49 launches
   US Air Force: 25 launches
   CASC: 16 launches
   🥇 Most active: RVSN USSR (789 launches)

📅 1980s (1980-1989):
   RVSN USSR: 416 launches
   General Dynamics: 42 launches
   NASA: 32 launches
   Martin Marietta: 29 launches
   Arianespace: 25 launches
   🥇 Most active: RVSN USSR (416 launches)

📅 Recent Years (2018-2020):
   2018: CASC - 37 launches
         Arianespace - 11 launches
         ULA - 8 launches
   2019: CASC - 27 launches
         Arianespace - 9 launches
         VKS RF - 8 launches
   2020: CASC - 19 launches
         Arianespace - 4 launches
         ULA - 4 launches

📊 All-Time Top 10 Organizations (Total Launches):
   1. RVSN USSR: 1,717 launches
   2. Arianespace: 279 launches
   3. General Dynamics: 249 launches
   4. CASC: 249 launches
   5. NASA: 203 launches
   6. VKS RF: 198 launches
   7. US Air Force: 157 

In [6]:
# Analysis: Can we fit a regression model to this data?

print("🔍 REGRESSION ANALYSIS EVALUATION\n")
print("="*70)

# Check what numeric variables we have
print("\n1️⃣ AVAILABLE NUMERIC VARIABLES:")
print("-" * 70)
numeric_cols = df_data.select_dtypes(include=[np.number]).columns.tolist()
print(f"   Numeric columns: {numeric_cols}")
print(f"   • Year (derived from Date)")
print(f"   • Price (USD Millions) - has {df_data['Price'].isna().sum():,} missing values")

# Check for potential relationships
print("\n2️⃣ POTENTIAL REGRESSION RELATIONSHIPS:")
print("-" * 70)

# Time-based relationships
print("\n   A. TIME-BASED REGRESSIONS (Year as predictor):")
print("      ✓ Year vs. Number of Launches")
print("      ✓ Year vs. Average Launch Price")
print("      ✓ Year vs. Failure Rate")
print("      → These show TRENDS but NOT linear relationships")

# Price-based relationships
print("\n   B. PRICE-BASED REGRESSIONS:")
df_with_price = df_data.dropna(subset=['Price', 'Year']).copy()
# Clean price column - remove commas and convert to numeric
df_with_price['Price'] = pd.to_numeric(df_with_price['Price'].astype(str).str.replace(',', ''), errors='coerce')
df_with_price = df_with_price.dropna(subset=['Price'])
print(f"      Available data points: {len(df_with_price):,} launches with price data")
print("      ✓ Year vs. Price")
print("      ✗ Organisation vs. Price (categorical)")
print("      ✗ Mission Status vs. Price (categorical)")

# Calculate correlations
print("\n3️⃣ CORRELATION ANALYSIS:")
print("-" * 70)

# Year vs Price correlation
if len(df_with_price) > 0:
    correlation = df_with_price['Year'].corr(df_with_price['Price'])
    print(f"   Year vs. Price correlation: {correlation:.4f}")
    if abs(correlation) < 0.3:
        strength = "WEAK"
    elif abs(correlation) < 0.7:
        strength = "MODERATE"
    else:
        strength = "STRONG"
    print(f"   Correlation strength: {strength}")

# Create scatter plot to visualize
print("\n4️⃣ VISUALIZATION:")
print("-" * 70)
fig = px.scatter(
    df_with_price,
    x='Year',
    y='Price',
    title='Scatter Plot: Year vs. Launch Price',
    labels={'Year': 'Year', 'Price': 'Price (USD Millions)'},
    opacity=0.5,
    height=500
)

# Add manual regression line
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(df_with_price['Year'], df_with_price['Price'])
years_range = np.array([df_with_price['Year'].min(), df_with_price['Year'].max()])
regression_line = slope * years_range + intercept

fig.add_scatter(
    x=years_range,
    y=regression_line,
    mode='lines',
    name=f'Linear Fit (R²={r_value**2:.3f})',
    line=dict(color='red', width=2, dash='dash')
)

fig.update_layout(
    template='plotly_white',
    hovermode='closest'
)

fig.show()

# Get regression statistics
if len(df_with_price) > 0:
    print(f"\n   Linear Regression Results:")
    print(f"   • Slope: {slope:.4f}")
    print(f"   • Intercept: {intercept:.4f}")
    print(f"   • R-squared: {r_value**2:.4f}")
    print(f"   • P-value: {p_value:.4e}")
    
    if r_value**2 < 0.1:
        fit_quality = "VERY POOR"
    elif r_value**2 < 0.3:
        fit_quality = "POOR"
    elif r_value**2 < 0.7:
        fit_quality = "MODERATE"
    else:
        fit_quality = "GOOD"
    print(f"   • Fit quality: {fit_quality} (R² = {r_value**2:.4f})")

print("\n" + "="*70)
print("🎯 FINAL ANSWER:")
print("="*70)

if len(df_with_price) > 10:
    if r_value**2 > 0.3:
        print("\n✅ YES - A regression model CAN be reasonably fit to this data")
        print("\nWHY:")
        print("   • We have continuous numeric variables (Year, Price)")
        print(f"   • Sufficient data points ({len(df_with_price):,} observations)")
        print(f"   • Moderate correlation (R² = {r_value**2:.4f})")
        print("   • Statistical significance (p < 0.05)" if p_value < 0.05 else "   ⚠️  Not statistically significant")
        print("\nBEST USE CASES:")
        print("   • Time series regression (Year vs. aggregated metrics)")
        print("   • Multiple regression with categorical encodings")
        print("   • Polynomial regression for non-linear trends")
    else:
        print("\n⚠️  PARTIALLY - Simple linear regression has LIMITED utility")
        print("\nWHY:")
        print(f"   • Weak correlation (R² = {r_value**2:.4f} < 0.3)")
        print("   • Data shows non-linear patterns/trends")
        print("   • High variance and outliers")
        print("   • Most relationships are TIME-TRENDS, not linear correlations")
        print("\nBETTER APPROACHES:")
        print("   • Time series analysis (ARIMA, seasonal decomposition)")
        print("   • Categorical analysis (group comparisons)")
        print("   • Polynomial or non-linear regression")
        print("   • Classification models (Success/Failure prediction)")
else:
    print("\n❌ NO - Insufficient data for meaningful regression")
    print(f"\nWHY: Only {len(df_with_price)} observations with complete data")

print("\n" + "="*70)


🔍 REGRESSION ANALYSIS EVALUATION


1️⃣ AVAILABLE NUMERIC VARIABLES:
----------------------------------------------------------------------
   Numeric columns: ['Year']
   • Year (derived from Date)
   • Price (USD Millions) - has 3,360 missing values

2️⃣ POTENTIAL REGRESSION RELATIONSHIPS:
----------------------------------------------------------------------

   A. TIME-BASED REGRESSIONS (Year as predictor):
      ✓ Year vs. Number of Launches
      ✓ Year vs. Average Launch Price
      ✓ Year vs. Failure Rate
      → These show TRENDS but NOT linear relationships

   B. PRICE-BASED REGRESSIONS:
      Available data points: 958 launches with price data
      ✓ Year vs. Price
      ✗ Organisation vs. Price (categorical)
      ✗ Mission Status vs. Price (categorical)

3️⃣ CORRELATION ANALYSIS:
----------------------------------------------------------------------
   Year vs. Price correlation: -0.3787
   Correlation strength: MODERATE

4️⃣ VISUALIZATION:
-------------------------------


   Linear Regression Results:
   • Slope: -8.7642
   • Intercept: 17748.9355
   • R-squared: 0.1434
   • P-value: 5.0496e-34
   • Fit quality: POOR (R² = 0.1434)

🎯 FINAL ANSWER:

⚠️  PARTIALLY - Simple linear regression has LIMITED utility

WHY:
   • Weak correlation (R² = 0.1434 < 0.3)
   • Data shows non-linear patterns/trends
   • High variance and outliers
   • Most relationships are TIME-TRENDS, not linear correlations

BETTER APPROACHES:
   • Time series analysis (ARIMA, seasonal decomposition)
   • Categorical analysis (group comparisons)
   • Polynomial or non-linear regression
   • Classification models (Success/Failure prediction)

